# Tweets data wrangling
#### Imports

In [2]:
import pandas as pd
import numpy as np
import requests
import os
import yaml
import json
import pprint
import tweepy
import time

## Gather
### Twitter Archive

In [3]:
# Create twitter archive DataFrame from the associated csv file
df_twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


### Tweets image prediction

In [4]:
image_pred_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file_name = image_pred_url.split('/')[-1]

if not os.path.isfile(file_name):
    # Retrieve the file content from the web
    response = requests.get(image_pred_url)
    # And save it in a tsv file
    with open(file_name, mode='wb') as file:
        file.write(response.content)
        
# Create tweet image prediction DataFrame from the downloaded tsv file
df_tweets_img_pred = pd.read_csv(file_name, sep='\t')
df_tweets_img_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### Tweeter API

In [5]:
# Load the API keys from the yaml file
with open('twitter_api_keys.yaml', mode='r') as file:
    twitter_cred = yaml.load(file, Loader=yaml.FullLoader)
    
consumer_key = twitter_cred['APIKEY']
consumer_secret = twitter_cred['APIKEYSECRET']
access_token = twitter_cred['ACCESSTOKEN']
access_secret = twitter_cred['ACCESSTOKENSECRET']

# Open a tweepy API object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

tweet_json_path = 'tweet_json.txt'
overwrite_json_file = False
if overwrite_json_file or not os.path.isfile(tweet_json_path):
    with open(tweet_json_path, 'w') as outfile:
        for idx, tweet_id in df_twitter_archive.tweet_id.iteritems():
            start_time = time.time()
            try:
                tweet_status = api.get_status(tweet_id, tweet_mode='extended')
                if idx > 0:
                    outfile.write("\n")
                json.dump(tweet_status._json, outfile)
                
                print("Tweet {} retrieved from Twitter API in {:.3f} seconds".format(tweet_id, time.time() - start_time))
            except tweepy.TweepError as e:
                print("Tweet {} couldn\'t be retrieved from Twitter API with error: \"{}\"".format(tweet_id, e))               

In [9]:
# Open the json txt file and extract the needed infos from the json data for each tweet saved
if os.path.isfile(tweet_json_path):
    with open(tweet_json_path, 'r') as json_file:
        # Create some columns for the new values retrieved from Twitter API
        df_twitter_archive['retweet_count'] = pd.Series(np.nan, dtype='Int64')
        df_twitter_archive['favorite_count'] = pd.Series(np.nan, dtype='Int64')
        
        # Iterate over each line in the txt file until the end of the file
        json_line = json_file.readline()
        while json_line:
            # Retrieve info from the json data structure
            tweet_json_data = json.loads(json_line)
            tweet_id = tweet_json_data['id']
            retweet_count = tweet_json_data['retweet_count']
            favorite_count = tweet_json_data['favorite_count']
            #print("Tweet {} has {} retweets and {} favorites".format(tweet_id,
            #                                                        retweet_count,
            #                                                        favorite_count))
            
            # And copy it in the Twitter archive DataFrame
            df_twitter_archive.loc[df_twitter_archive.tweet_id == tweet_id, 'retweet_count'] = retweet_count
            df_twitter_archive.loc[df_twitter_archive.tweet_id == tweet_id, 'favorite_count'] = favorite_count
            
            json_line = json_file.readline()

In [7]:
df_twitter_archive.info()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,retweet_count,favorite_count
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None,<NA>,<NA>
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None,<NA>,<NA>
101,872668790621863937,NaN,NaN,2017-06-08 04:17:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @loganamnosis: Penelope here is doing me qu...,8.726576e+17,1.547674e+08,2017-06-08 03:32:35 +0000,https://twitter.com/loganamnosis/status/872657...,14,10,None,None,None,None,None,<NA>,<NA>
104,872261713294495745,NaN,NaN,2017-06-07 01:19:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Harry. His ears are activated one at a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/872261713...,13,10,Harry,None,None,None,None,<NA>,<NA>
118,869988702071779329,NaN,NaN,2017-05-31 18:47:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: We only rate dogs. This is quit...,8.591970e+17,4.196984e+09,2017-05-02 00:04:57 +0000,https://twitter.com/dog_rates/status/859196978...,12,10,quite,None,None,None,None,<NA>,<NA>
132,866816280283807744,NaN,NaN,2017-05-23 00:41:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Jamesy. He gives a kiss...,8.664507e+17,4.196984e+09,2017-05-22 00:28:40 +0000,https://twitter.com/dog_rates/status/866450705...,13,10,Jamesy,None,None,pupper,None,<NA>,<NA>
155,861769973181624320,NaN,NaN,2017-05-09 02:29:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: ""Good afternoon class today we'...",8.066291e+17,4.196984e+09,2016-12-07 22:38:52 +0000,https://twitter.com/dog_rates/status/806629075...,13,10,None,None,None,None,None,<NA>,<NA>
182,856602993587888130,NaN,NaN,2017-04-24 20:17:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Luna. It's her first ti...,8.447048e+17,4.196984e+09,2017-03-23 00:18:10 +0000,https://twitter.com/dog_rates/status/844704788...,13,10,Luna,None,None,None,None,<NA>,<NA>
211,851953902622658560,NaN,NaN,2017-04-12 00:23:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Astrid. She's a guide d...,8.293743e+17,4.196984e+09,2017-02-08 17:00:26 +0000,https://twitter.com/dog_rates/status/829374341...,13,10,Astrid,doggo,None,None,None,<NA>,<NA>
247,845459076796616705,NaN,NaN,2017-03-25 02:15:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Here's a heartwarming scene of ...,7.562885e+17,4.196984e+09,2016-07-22 00:43:32 +0000,https://twitter.com/dog_rates/status/756288534...,12,10,None,None,None,None,None,<NA>,<NA>


## Assess

## Clean

## Analyze